Reproducing [Interpreting Context Look-ups in Transformers: Investigating Attention-MLP Interactions](https://arxiv.org/abs/2402.15055)

In [1]:
print("Init Reproducing Neo et al. 2024")

Init Reproducing Neo et al. 2024


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

/Users/kyleng/B_Organized/H_Passion_Projects/mech_interp/reproducing-neo-et-al-2024/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_device():
    if torch.cuda.is_available():
        print("Using CUDA GPU")
        return torch.device("cuda")
    else:
        print("Using CPU")
        return torch.device("cpu")
device = get_device()

Using CPU


In [17]:
tokenizer = AutoTokenizer.from_pretrained(
    "openai-community/gpt2",
    output_hidden_states=True,
    output_attentions=True
)
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

In [24]:
print("Sanity Check that model is loaded: ")
tokenized_sample_input = tokenizer("one two three", return_tensors="pt")

with torch.no_grad():
    model_output = model(**tokenized_sample_input)

next_token_id = model_output.logits[:, -1, :].argmax(dim=-1)

print("Tokenization shape", tokenized_sample_input.input_ids.shape)
print("Model Output shape", model_output.logits.shape)

print("Predicted next token:", tokenizer.decode(next_token_id))

Sanity Check that model is loaded: 
Tokenization shape torch.Size([1, 3])
Model Output shape torch.Size([1, 3, 50257])
Predicted next token:  four
